Дорогой студент!

В данном домашнем задании предоставляется возможность поработать с задачей по распознаванию позитивных и негативных отзывы людей по автомобилю Tesla. База для обучения содержит два текстовых файла с рядом строчных отзывов с мнением людей об автомобиле Tesla, соответственно негативного и позитивного содержания. Ссылка на скачивание базы уже включена в ноутбук задания.


Необходимо выполнить следующие действия:

  1. Загрузите саму базу по ссылке и подговьте файлы базы для обработки.
  2. Создайте обучающую и проверочную выборки, обратив особое внимание на балансировку базы: количество примеров каждого класса должно быть примерно одного порядка.
  3. Подготовьте выборки для обучения и обучите сеть. Добейтесь результата точности сети в 85-90% на проверочной выборке.
   


**Импорт библиотек, загрузка базы и подготовка её к предобработке.**

In [ ]:
# Работа с массивами данных
import numpy as np

# Работа с таблицами
import pandas as pd

# Отрисовка графиков
import matplotlib.pyplot as plt

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Заполнение последовательностей до определенной длины
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Загрузка датасетов из облака google
import gdown

# Для работы с файлами в Colaboratory
import os

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/tesla.zip', None, quiet=True)

'tesla.zip'

**Деление общей базы отзывов на данные для обучения и проверки.**

In [ ]:
# Распаковка архива в папку tesla
!unzip -qo tesla.zip -d tesla/

# Просмотр содержимого папки
!ls tesla

'Негативный отзыв.txt'	'Позитивный отзыв.txt'


**Загрузка текста из файлов, преобразование в одну строку.**

In [ ]:
# Объявляем функции для чтения файла. На вход отправляем путь к файлу
def read_text(file_name):

  # Задаем открытие нужного файла в режиме чтения
  read_file = open(file_name, 'r')

  # Читаем текст
  text = read_file.read()

  # Переносы строки переводим в пробелы
  text = text.replace("\n", " ")

  # Возвращаем текст файла
  return text

# Объявляем интересующие нас классы
class_names = ["Негативный отзыв", "Позитивный отзыв"]

# Считаем количество классов
num_classes = len(class_names)

**Тексты в один список.**

In [ ]:
import os
# Создаём список под тексты для обучающей выборки
texts_list = []

# Циклом проводим итерацию по текстовым файлам в папке отзывов
for j in os.listdir('/content/tesla/'):

  # Добавляем каждый файл в общий список для выборки
        texts_list.append(read_text('/content/tesla/' + j))

        # Выводим на экран сообщение о добавлении файла
        print(j, 'добавлен в обучающую выборку')

Негативный отзыв.txt добавлен в обучающую выборку
Позитивный отзыв.txt добавлен в обучающую выборку


In [ ]:
# Узнаем объём каждого текста в словах и символах
texts_len = [len(text) for text in texts_list]

# Устанавливаем "счётчик" номера текста
t_num = 0

# Выводим на экран  информационное сообщение
print(f'Размеры текстов по порядку (в токенах):')

# Циклом проводим итерацию по списку с объёмами текстов
for text_len in texts_len:

  # Запускаем "счётчик" номера текста
  t_num += 1

  # Выводим на экран сообщение о номере и объёме текста
  print(f'Текст №{t_num}: {text_len}')

Размеры текстов по порядку (в токенах):
Текст №1: 134535
Текст №2: 213381


Далее рассчитаем, сколько символов составит 80% объёма каждого текста, чтобы по полученному индексу отделить эти 80% на обучающую и оставшиеся 20% на проверочную выборку. Эти значения необходимы для подготовки деления на выборки слайсингом по индексу.

In [ ]:
# Создаём список с вложенным циклом по длинам текстов, где i - 100% текста, i/5 - 20% текста
train_len_shares = [(i - round(i/5)) for i in texts_len]

# Устанавливаем "счётчик" номера текста
t_num = 0

# Циклом проводим итерацию по списку с объёмами текстов равными 80% от исходных
for train_len_share in train_len_shares:

  # Запускаем "счётчик" номера текста
  t_num += 1

  # Выводим на экран сообщение о номере и объёме текста в 80% от исходного
  print(f'Доля 80% от текста №{t_num}: {train_len_share} символов')

Доля 80% от текста №1: 107628 символов
Доля 80% от текста №2: 170705 символов


Импортируем функцию **chain()** для добавления текстов в каждую выборку.

---
 Дополнительная информация: ([База знаний УИИ  - **"Методы работы со списками: функция chain( )**"](https://colab.research.google.com/drive/1KJKg_WYD8Vq63cciOMBEEAhFpyPv0A0V?usp=sharing/))

---

Производём нарезку (метод слайсинга) по полученному ранее индексу для формирования текстов отдельно для обучающей(80%) и проверочной(20%) выборок:

In [ ]:
print(len(texts_list))

2


In [ ]:
from itertools import chain
# Ваше решение

train_text = list(chain((texts_list[0][:train_len_shares[0]], texts_list[1][:train_len_shares[1]])))
test_text = list(chain((texts_list[0][train_len_shares[0]:], texts_list[1][train_len_shares[1]:])))
print(len(train_text))
print(len(test_text))

2
2


In [ ]:
train_text[0]

'После 170 на трассе она чувствует себя неуверенно.\xa0 Кстати сидения очень удобными тоже не назовёшь. ... Это моё личное впечатление об автомобиле. Шляпа, а не авто, все не продумано до мелочей. Пока это гаджет а не авто. Это не авто, а бренд. Стремнные материалы, стремно все. Прям фу фу фу и дизайн снаружи и внутри. Продал\xa0 слава богу, экран менять целое дело через полтора года , негодование обычно возникает из-за несостоявшихся обманутых ожиданий. Не нужно воспринимать теслу как эталон, верх совершенства и т.п. Относитесь к тесле как гаджету и тогда не будет несбывшихся ожиданий от него гаджету можно простить и заоблачную цену при скромных характеристиках и зазоры на дверях при выдвигающихся ручках дверей. Походу ещё один бот и видите как тезисы выкидывает не нужно воспринимать Теслу как эталон, Тесла это гаджет, который нет смысла покупать...\xa0 Как я уже и писал к предыдущему посту КАЧЕСТВО СБОРКИ ХРОМАЕТ! Я пару дней назад указал на не совсем качественную подгонку деталей Mo

In [ ]:
train_text[1]

' Водитель наслаждается от такого авто и вождения  Красивая машина нужно брать  Тесла топ, тащусь от этой тачки Плюсы - зарядка бесплатная.\xa0 Машина топ Технологии по автоматизации не стоят на месте! Красивая машина нужно брать  Хочу радоваться езде на своей тесле!\xa0  Это вам не приора\xa0  Автомобиль - мечта!!!\xa0  Эта машина моя мечта  Тесла это гаджет, который нет смысла покупать? Сотни тысяч людей уже нашли смысл купить) Ну наконец любители трона объявились) а то будто я один от него фанатею  , как раз таки на этой машине он всё сделал так, как будто оно было с завода. С таким интерьером прям очень хорошо смотрится. Напоминает подсветку, как на каком-нибудь мерседесе. А пацанские девяточки здесь и рядом не стоят, потому что с их древним салоном сразу видно, что никакой подсветки там никогда не было и её тупо вколхозили.  Образно говоря - мой IPS дисплей на ноутбуке со светодиодной подсветкой и разрешением 2048*1536 точек с соотношением сторон 4:3 настолько офуительно хорош, чт

In [ ]:
VOCAB_SIZE = 10000
WIN_SIZE   = 1000
WIN_HOP    = 100

tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

tokenizer.fit_on_texts(train_text)
tokenizer.fit_on_texts(test_text)

In [ ]:
seq_train = tokenizer.texts_to_sequences(train_text)
seq_test = tokenizer.texts_to_sequences(test_text)

In [ ]:
def split_sequence(sequence,   # Последовательность индексов
                   win_size,   # Размер окна для деления на примеры
                   hop):       # Шаг окна

    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

In [ ]:
def vectorize_sequence(seq_list,    # Список последовательностей индексов
                       win_size,    # Размер окна для деления на примеры
                       hop):        # Шаг окна

    # В списке последовательности следуют в порядке их классов (их кол-во сповпадает с кол-вом классов)
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):

        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)

        # Добавление отрезков в выборку

        x += vectors

        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

In [ ]:
x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP)
x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(388, 2000) (388, 2)
(70, 2000) (70, 2)


In [ ]:
x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

In [ ]:
model_BoW = Sequential()                                            # Создание последовательной модели нейросети
model_BoW.add(Dense(184, input_dim=VOCAB_SIZE, activation="relu"))  # Первый полносвязный слой
model_BoW.add(Dense(86, activation="relu"))                         # Второй полносвязный слой
model_BoW.add(Dense(33, activation="relu"))                         # Третий полносвязный слой
model_BoW.add(BatchNormalization())                                 # Слой пакетной нормализации
model_BoW.add(Dropout(0.1))                                         # Слой регуляризации Dropout
model_BoW.add(Dense(num_classes, activation='sigmoid'))             # Выходной полносвязный слой

model_BoW.compile(optimizer='adam',                                 # Компиляция модели для обучения на данных вида Bag of Words
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_BoW.fit(x_train_01,                            # Обучающая выборка Bag of Words
                        y_train,                               # Метки классов обучающей выборки
                        epochs=20,                             # Количество эпох
                        batch_size=32,                         # Размер подвыборки для одного шага по данным на эпохе
                        validation_data=(x_test_01, y_test))

Epoch 1/20
13/13 [==============================] - 2s 45ms/step - loss: 0.1903 - accuracy: 0.9191 - val_loss: 0.4432 - val_accuracy: 0.6222
Epoch 2/20
13/13 [==============================] - 0s 35ms/step - loss: 0.0277 - accuracy: 1.0000 - val_loss: 0.5624 - val_accuracy: 0.6222
Epoch 3/20
13/13 [==============================] - 0s 37ms/step - loss: 0.0169 - accuracy: 1.0000 - val_loss: 0.6062 - val_accuracy: 0.6222
Epoch 4/20
13/13 [==============================] - 0s 36ms/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.5567 - val_accuracy: 0.6222
Epoch 5/20
13/13 [==============================] - 0s 35ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.4951 - val_accuracy: 0.6222
Epoch 6/20
13/13 [==============================] - 0s 36ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.4365 - val_accuracy: 0.6444
Epoch 7/20
13/13 [==============================] - 0s 36ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.3966 - val_accuracy: 0.7222
Epoch 8/20
13